In [1]:
!python -V

Python 3.10.13


In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics 
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import plot_tree

from joblib import dump

import seaborn as sns
import matplotlib.pyplot as plt

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope

import boto3

In [3]:
import mlflow
import os

In [4]:
random_state = 42

# Configuring Mlflow

In [5]:
os.environ["AWS_PROFILE"] = "mlflow-profile"
TRACKING_SERVER_HOST = "ec2-3-140-187-200.us-east-2.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment("diabetes-uci")

<Experiment: artifact_location='s3://mlflow-artifacts-bucket-aalva/1', creation_time=1721663385991, experiment_id='1', last_update_time=1721663385991, lifecycle_stage='active', name='diabetes-uci', tags={}>

In [6]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-3-140-187-200.us-east-2.compute.amazonaws.com:5000'


# Loading data

In [7]:
data_path = '../data/raw/diabetes.csv'
def load_data(data_path):
    diabetes_df = pd.read_csv(data_path)
    
    diabetes_df = diabetes_df.copy(deep=True)
    diabetes_df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = diabetes_df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.NaN)

    diabetes_df['Glucose'] = diabetes_df['Glucose'].fillna(diabetes_df['Glucose'].mean())
    diabetes_df['BloodPressure'] = diabetes_df['BloodPressure'].fillna(diabetes_df['BloodPressure'].mean())
    diabetes_df['SkinThickness'] = diabetes_df['SkinThickness'].fillna(diabetes_df['SkinThickness'].median())
    diabetes_df['Insulin'] = diabetes_df['Insulin'].fillna(diabetes_df['Insulin'].median())
    diabetes_df['BMI'] = diabetes_df['BMI'].fillna(diabetes_df['BMI'].median())

    return diabetes_df


# Training Random Forest

In [8]:
def train(data_path, max_depth, max_features, n_estimators):

    data_df = load_data(data_path)

    X = data_df.drop('Outcome', axis=1)
    y = data_df['Outcome']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                        random_state=random_state,
                                                       stratify=y)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)

    rf_model = RandomForestClassifier(
        max_depth=max_depth, max_features=max_features, n_estimators=n_estimators
    )
    rf_model.fit(X_train, y_train)
    
    y_pred = rf_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)

    mlparams = {
        'max_depth': max_depth,
        'max_features': max_features,
        'n_estimators': n_estimators
    }
    mlflow.log_params(mlparams)
    mlflow.set_tag('model', 'random-forest')

    mlmetrics = {'accuracy': acc}
    print(f'accuracy: {acc}')
    mlflow.log_metrics(mlmetrics)

    return rf_model

In [10]:
max_depth = 8
max_features = 0.75
n_estimators = 200

with mlflow.start_run():
    train(data_path, max_depth, max_features, n_estimators)

accuracy: 0.7467532467532467


# Hyperparameter Optimization

In [11]:
data_df = load_data(data_path)

In [17]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("developer", "alexis")
        mlflow.set_tag("model", "random_forest")
        mlflow.log_param('train_data_path', f'{data_path}')
        
        X = data_df.drop('Outcome', axis=1)
        y = data_df['Outcome']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                            random_state=random_state,
                                                            stratify=y)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
    
        rf_model = RandomForestClassifier(**params)
        rf_model.fit(X_train_scaled, y_train)
        
        y_pred = rf_model.predict(X_test_scaled)
        acc = accuracy_score(y_test, y_pred)

        mlmetrics = {'accuracy': acc}
        print(f'accuracy: {acc}')
        mlflow.log_metrics(mlmetrics)
    
    return {'loss': -acc, 'status': STATUS_OK, 'params': params}

In [35]:
# Define the search space
search_space = {
    'max_depth': scope.int(hp.choice('max_depth', [10, 20, 30])),
    'max_features': hp.choice('max_features', ['sqrt','log2']),
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 300, 50)),
}

In [36]:
trials = Trials()
best = fmin(fn=objective,
            space=search_space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

# Extract the best parameters
best_params = trials.best_trial['result']['params']

accuracy: 0.7532467532467533                                                                                                            
accuracy: 0.7532467532467533                                                                                                            
accuracy: 0.7467532467532467                                                                                                            
accuracy: 0.7467532467532467                                                                                                            
accuracy: 0.7272727272727273                                                                                                            
accuracy: 0.7337662337662337                                                                                                            
accuracy: 0.7337662337662337                                                                                                            
accuracy: 0.7207792207792207             

# Save best model

In [37]:
artifact_path = '../models/random_forest.joblib'

with mlflow.start_run():
    mlflow.log_params(best_params)
    mlflow.set_tag("developer", "alexis")
    mlflow.set_tag("model", "random_forest")
    mlflow.log_param('train_data_path', f'{data_path}')
    
    print(f'Best params: {best_params}')
    # Retrain the model with the best parameters
    data_df = load_data(data_path)
    X = data_df.drop('Outcome', axis=1)
    y = data_df['Outcome']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                        random_state=random_state,
                                                        stratify=y)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    best_model = RandomForestClassifier(**best_params)
    best_model.fit(X_train_scaled, y_train)

    y_pred = best_model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)

    mlmetrics = {'accuracy': acc}
    print(f'accuracy: {acc}')
    mlflow.log_metrics(mlmetrics)
    
    mlflow.sklearn.log_model(
            best_model,
            artifact_path=artifact_path,
        )

Best params: {'max_depth': 10, 'max_features': 'log2', 'n_estimators': 50}
accuracy: 0.7402597402597403


/home/codespace/.local/share/virtualenvs/mlops-diabetes--DZkgjQj/lib/python3.10/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/codespace/.local/share/virtualenvs/mlops-diabetes--DZkgjQj/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


ProfileNotFound: The config profile (mlflow-profile) could not be found